In [1]:
import pandas as pd
import numpy as np
import random

In [3]:
song_path ="spotify-2023 2.csv"# "/Users/kenny/Desktop/537project/spotify-2023 2.csv"
song_raw = pd.read_csv(song_path, encoding = 'latin1')
song_raw.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [4]:
user_path = "/Users/kenny/Desktop/537project/Spotify_data.xlsx"
user_raw = pd.read_excel(user_path)
user_raw.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kenny/Desktop/537project/Spotify_data.xlsx'

In [5]:
# Drop NaNs from song raw, 817 rows left
song_df = song_raw.dropna()
# Create a column of release date
song_df = song_df.rename(columns = {"released_year" : "year", "released_month" : "month", "released_day" : "day"})
song_df["released_date"] = pd.to_datetime(song_df[["year", "month", "day"]])
# Create a column of song_id 000-817
song_df["song_id"] = [str(i).zfill(len(str(len(song_df)))) for i in range(len(song_df))]
song_df = song_df.drop(["year", "month", "day"], axis = 1)
song_df = song_df.drop(["artist_count", "in_spotify_playlists", 
                        "in_spotify_charts", "in_apple_playlists", 
                        "in_apple_charts", "in_deezer_playlists", 
                        "in_deezer_charts", "in_shazam_charts", "mode"], axis = 1)
# Create a column of promotion, half songs are promoted
song_df["promotion"] = random.choices([0, 1], k = len(song_df), weights = [0.25, 0.75])
song_df

,track_name,artist(s)_name,streams,bpm,key,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_date,song_id,promotion
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",141381703,125,B,80,89,83,31,0,8,4,2023-07-14,000,1
1,LALA,Myke Towers,133716286,92,C#,71,61,74,7,0,10,4,2023-03-23,001,1
2,vampire,Olivia Rodrigo,140003974,138,F,51,32,53,17,0,31,6,2023-06-30,002,1
3,Cruel Summer,Taylor Swift,800840817,170,A,55,58,72,11,0,11,15,2019-08-23,003,1
4,WHERE SHE GOES,Bad Bunny,303236322,144,A,65,23,80,14,63,11,6,2023-05-18,004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,91473363,144,A,60,24,39,57,0,8,3,2022-11-03,812,1
949,Bigger Than The Whole Sky,Taylor Swift,121871870,166,F#,42,7,24,83,1,12,6,2022-10-21,813,1
950,A Veces (feat. Feid),"Feid, Paulo Londra",73513683,92,C#,80,81,67,4,0,8,6,2022-11-03,814,0
951,En La De Ella,"Feid, Sech, Jhayco",133895612,97,C#,82,67,77,8,0,12,5,2022-10-20,815,1


In [6]:
artist_df = pd.DataFrame()
# Create artist_name column for artist
artist_df["artist_name"] = song_df["artist(s)_name"].unique()
# For rows that contains more than 1 names, separate them to multiple rows
artist_df["artist_name"] = artist_df["artist_name"].str.split(", ")
artist_df = artist_df.explode("artist_name")
# Strip useless spaces
artist_df["artist_name"] = artist_df["artist_name"].str.strip()
# Delete duplicate artist names
artist_df = artist_df.drop_duplicates()
# Create a column of artist_id 000-628
artist_df["artist_id"] = [str(i).zfill(len(str(len(artist_df)))) for i in range(len(artist_df))]
artist_df.head()

,artist_name,artist_id
0,Latto,000
0,Jung Kook,001
1,Myke Towers,002
2,Olivia Rodrigo,003
3,Taylor Swift,004


In [7]:
# Create song_artist table
song_artist_df = song_df.loc[:, ["song_id", "artist(s)_name"]]
# For songs with multiple artists, separate them to multiple rows
song_artist_df["artist(s)_name"] = song_artist_df["artist(s)_name"].str.split(", ")
song_artist_df = song_artist_df.explode("artist(s)_name")
song_artist_df = song_artist_df.drop_duplicates()
# Left join artist table
song_artists_df = pd.merge(song_artist_df, artist_df, left_on = "artist(s)_name", right_on = "artist_name", how = "left")
song_artists_df = song_artists_df.loc[:, ["song_id", "artist_id"]].drop_duplicates()
song_artists_df.head()
#song_artists_df.shape

,song_id,artist_id
0,000,000
1,000,001
2,001,002
3,002,003
4,003,004


In [10]:
user_path ='Spotify_data.xlsx' #"/Users/kenny/Desktop/537project/Spotify_data.xlsx"
user_raw = pd.read_excel(user_path)
user_raw.head()

,Age,Gender,spotify_usage_period,spotify_listening_device,spotify_subscription_plan,premium_sub_willingness,preffered_premium_plan,preferred_listening_content,fav_music_genre,music_time_slot,music_Influencial_mood,music_lis_frequency,music_expl_method,music_recc_rating,pod_lis_frequency,fav_pod_genre,preffered_pod_format,pod_host_preference,preffered_pod_duration,pod_variety_satisfaction
0,20-35,Female,More than 2 years,Smart speakers or voice assistants,Free (ad-supported),Yes,Family Plan-Rs 179/month,Podcast,Melody,Night,Sadness or melancholy,leisure time,Playlists,3,Daily,Comedy,Interview,Both,Both,Ok
1,12-20,Male,More than 2 years,Computer or laptop,Free (ad-supported),Yes,Individual Plan- Rs 119/ month,Podcast,Rap,Afternoon,Social gatherings or parties,Workout session,Playlists,2,Several times a week,Comedy,Interview,Both,None,Satisfied
2,35-60,Others,6 months to 1 year,Smart speakers or voice assistants,Free (ad-supported),Yes,Student Plan-Rs 59/month,Podcast,Pop,Night,Relaxation and stress relief,"Study Hours, While Traveling",Playlists,4,Once a week,Sports,Interview,None,Both,Satisfied
3,20-35,Female,1 year to 2 years,"Smartphone, Smart speakers or voice assistants",Free (ad-supported),No,None,Music,Melody,Night,"Relaxation and stress relief, Social gathering...","Office hours, Workout session, leisure time","recommendations, Playlists",4,Never,None,None,None,None,Ok
4,20-35,Female,1 year to 2 years,Smartphone,Free (ad-supported),No,None,Music,Melody,Night,Relaxation and stress relief,leisure time,"recommendations, Playlists",4,Rarely,Lifestyle and Health,Story telling,Well known individuals,Both,Ok


In [11]:
user_df = user_raw.loc[:, ["Age", "Gender", "spotify_usage_period", "spotify_subscription_plan",
                          "music_lis_frequency", "fav_music_genre", "music_Influencial_mood"]]
user_df = user_df.dropna()
user_df = user_df.rename(columns = {"spotify_usage_period" : "usage_period",
                                   "spotify_subscription_plan" : "plan",
                                   "music_lis_frequency" : "listening_scenario",
                                   "music_influential_mood" : "mood"})
user_df["user_id"] = [str(i).zfill(len(str(len(user_df)))) for i in range(len(user_df))]
user_df["password"] = "000000"
user_df

,Age,Gender,usage_period,plan,listening_scenario,fav_music_genre,music_Influencial_mood,user_id,password
0,20-35,Female,More than 2 years,Free (ad-supported),leisure time,Melody,Sadness or melancholy,000,000000
1,12-20,Male,More than 2 years,Free (ad-supported),Workout session,Rap,Social gatherings or parties,001,000000
2,35-60,Others,6 months to 1 year,Free (ad-supported),"Study Hours, While Traveling",Pop,Relaxation and stress relief,002,000000
3,20-35,Female,1 year to 2 years,Free (ad-supported),"Office hours, Workout session, leisure time",Melody,"Relaxation and stress relief, Social gathering...",003,000000
4,20-35,Female,1 year to 2 years,Free (ad-supported),leisure time,Melody,Relaxation and stress relief,004,000000
...,...,...,...,...,...,...,...,...,...
515,20-35,Female,More than 2 years,Free (ad-supported),While Traveling,Pop,"Uplifting and motivational, Sadness or melancholy",515,000000
516,20-35,Female,More than 2 years,Free (ad-supported),"While Traveling, Workout session, leisure time",Pop,"Relaxation and stress relief, Uplifting and mo...",516,000000
517,20-35,Female,More than 2 years,Free (ad-supported),leisure time,Pop,Relaxation and stress relief,517,000000
518,20-35,Female,6 months to 1 year,Premium (paid subscription),"Workout session, leisure time",Melody,"Relaxation and stress relief, Uplifting and mo...",518,000000


In [12]:
song_df

,track_name,artist(s)_name,streams,bpm,key,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_date,song_id,promotion
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",141381703,125,B,80,89,83,31,0,8,4,2023-07-14,000,1
1,LALA,Myke Towers,133716286,92,C#,71,61,74,7,0,10,4,2023-03-23,001,1
2,vampire,Olivia Rodrigo,140003974,138,F,51,32,53,17,0,31,6,2023-06-30,002,1
3,Cruel Summer,Taylor Swift,800840817,170,A,55,58,72,11,0,11,15,2019-08-23,003,1
4,WHERE SHE GOES,Bad Bunny,303236322,144,A,65,23,80,14,63,11,6,2023-05-18,004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,91473363,144,A,60,24,39,57,0,8,3,2022-11-03,812,1
949,Bigger Than The Whole Sky,Taylor Swift,121871870,166,F#,42,7,24,83,1,12,6,2022-10-21,813,1
950,A Veces (feat. Feid),"Feid, Paulo Londra",73513683,92,C#,80,81,67,4,0,8,6,2022-11-03,814,0
951,En La De Ella,"Feid, Sech, Jhayco",133895612,97,C#,82,67,77,8,0,12,5,2022-10-20,815,1


In [15]:
user_df

,Age,Gender,usage_period,plan,listening_scenario,fav_music_genre,music_Influencial_mood,user_id,password
0,20-35,Female,More than 2 years,Free (ad-supported),leisure time,Melody,Sadness or melancholy,000,000000
1,12-20,Male,More than 2 years,Free (ad-supported),Workout session,Rap,Social gatherings or parties,001,000000
2,35-60,Others,6 months to 1 year,Free (ad-supported),"Study Hours, While Traveling",Pop,Relaxation and stress relief,002,000000
3,20-35,Female,1 year to 2 years,Free (ad-supported),"Office hours, Workout session, leisure time",Melody,"Relaxation and stress relief, Social gathering...",003,000000
4,20-35,Female,1 year to 2 years,Free (ad-supported),leisure time,Melody,Relaxation and stress relief,004,000000
...,...,...,...,...,...,...,...,...,...
515,20-35,Female,More than 2 years,Free (ad-supported),While Traveling,Pop,"Uplifting and motivational, Sadness or melancholy",515,000000
516,20-35,Female,More than 2 years,Free (ad-supported),"While Traveling, Workout session, leisure time",Pop,"Relaxation and stress relief, Uplifting and mo...",516,000000
517,20-35,Female,More than 2 years,Free (ad-supported),leisure time,Pop,Relaxation and stress relief,517,000000
518,20-35,Female,6 months to 1 year,Premium (paid subscription),"Workout session, leisure time",Melody,"Relaxation and stress relief, Uplifting and mo...",518,000000


In [13]:
#song recommendation
def song_recommendation(user_id,song_df,user_df):
    
    def get_user_favorite_genre(user_id,user_df):
        user_info = user_df[user_df['user_id'] == user_id]
        if user_info.empty:
            return f"User with ID {user_id} not found."
        favorite_genre = user_info['fav_music_genre'].values[0]
        return favorite_genre



    def rec_songs(min_value, max_value,song_df):
        filtered_songs = song_df[(song_df['criteria'] >= min_value) & (song_df['criteria'] <= max_value)]
        random_5 = filtered_songs.sample(n=5, random_state=42)
        selected_song = random_5['track_name'].tolist()
        return selected_song

    
    
    def rec_songs_by_release_date(song_df,date):
        song_df['released_date'] = pd.to_datetime(song_df['released_date'])
        filtered_songs = song_df[song_df['released_date'] >= date]
        random_5 = filtered_songs.sample(n=5, random_state=42)
        selected_songs = random_5['track_name'].tolist() 
        return selected_songs

    #all songs
    temp_song_df = song_df.copy()
    temp_song_df['criteria'] = temp_song_df['danceability_%'] + temp_song_df['energy_%'] 
    
   
    #free songs
    temp_song_df2 = temp_song_df.copy()
    temp_song_df2 = temp_song_df2[temp_song_df2['promotion'] == 1]

    
    if (user_id not in user_df['user_id'].values):
        print(f"User {user_id} does not exist.")
        return None
    
    #for free users
    elif (user_df[user_df['user_id'] == user_id]['plan'].values == 'Free (ad-supported)'):
        
        song_list1 = []
        if (get_user_favorite_genre(user_id,user_df)=='classical'):
             song_list1=rec_songs(0,25,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Classical & melody, dance'):
             song_list1=rec_songs(0,80,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Old songs'):
             song_list1=rec_songs(0,50,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Melody'):
             song_list1=rec_songs(41,80,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Pop'):
             song_list1=rec_songs(81,120,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Kpop'):
             song_list1=rec_songs(81,100,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Electronic/Dance'):
             song_list1=rec_songs(81,120,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Rap'):
             song_list1=rec_songs(81,130,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Rock'):
             song_list1=rec_songs(100,152,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='trending songs random'):
             song_list1=rec_songs_by_release_date(temp_song_df2, date='2023-01-01')
        else:
             song_list1=rec_songs(0,152,temp_song_df2)
        return  song_list1
        

    #for premium users
    else:
        song_list = []
        if (get_user_favorite_genre(user_id,user_df)=='classical'):
             song_list=rec_songs(0,25,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Classical & melody, dance'):
             song_list=rec_songs(0,80,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Old songs'):
             song_list=rec_songs(0,50,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Melody'):
             song_list=rec_songs(41,80,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Pop'):
             song_list=rec_songs(81,120,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Kpop'):
             song_list=rec_songs(81,100,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Electronic/Dance'):
             song_list=rec_songs(81,120,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Rap'):
             song_list=rec_songs(81,130,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Rock'):
             song_list=rec_songs(100,152,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='trending songs random'):
             song_list=rec_songs_by_release_date(temp_song_df, date='2023-01-01')
        else:
             song_list=rec_songs(0,152,temp_song_df)
        return  song_list
   

In [19]:
#test
song_recommendation('500',song_df,user_df)

['Down Under (feat. Colin Hay)',
 "Say You Won't Let Go",
 'Monï¿½ï¿½y so',
 'See You Again',
 'Never Felt So Alone']